# python多进程编程

python受GIL限制,无法利用到多核,要使用多核提高cpu的利用率这种时候最简单的方式就是使用多进程实现突破GIL限制.

换言之python多进程的价值体现在CPU密集型作业上.

进程(Process)是计算机中的程序关于某数据集合上的一次运行活动，是系统进行资源分配和调度的基本单位，是操作系统结构的基础。在早期面向进程设计的计算机结构中，进程是程序的基本执行实体；在当代面向线程设计的计算机结构中，进程是线程的容器。程序是指令、数据及其组织形式的描述，进程是程序的实体.


## python调用系统fork

Unix/Linux操作系统提供了一个fork()系统调用.普通的函数调用，调用一次，返回一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程），然后，分别在父进程和子进程内返回。

子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。

python`os`模块封装了`fork()`(unix-like系统).事实上在类unix系统下,python的多进程都是基于fork的,而windows下情况就不一样了.

## windows下的多进程局限性

在windows下,由于没有fork,python的多进程模块`multiprocessing`必须要有`if __name__=='__main__':`也就是说它无法在非入口模块下使用


In [1]:
import os
print('Process ({}) 开始...'.format(os.getpid()))
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('子进程: ({}) 它的父进程是: ({}).'.format(os.getpid(), os.getppid()))
else:
    print('父进程 ({}) 产生了子进程: ({}).'.format(os.getpid(), pid))


Process (1372) 开始...
父进程 (1372) 产生了子进程: (1377).
子进程: (1377) 它的父进程是: (1372).


## 使用concurrent.futures进行高层抽象的多进程操作

在python3中,模块`concurrent.futures`提供了一些更加简单易用的多进程操作,它主要利用进程池.

这个库支持多线程和多进程,接口一样,只是使用的对象不同而已.

`concurrent.futures`提供两种编程模型:

+ 并行任务模型

    单独任务独立使用自己的过程和数据,多任务独立并行计算

+ MapReduce模型

    为各个进程分发数据执行相同的过程

### 并行任务模型

这个模型使用`submit`提交任务到上下文管理器,之后使用返回对象的`result()`方法阻塞io等待任务完成


In [2]:
from concurrent.futures import ProcessPoolExecutor,as_completed
from random import randrange
from time import time

In [3]:
def arcfour(key, in_bytes, loops=20):
    """rc4算法"""
    kbox = bytearray(256)  # create key box
    for i, car in enumerate(key):  # copy key and vector
        kbox[i] = car
    j = len(key)
    for i in range(j, 256):  # repeat until full
        kbox[i] = kbox[i-j]

    # [1] initialize sbox
    sbox = bytearray(range(256))

    # repeat sbox mixing loop, as recommened in CipherSaber-2
    # http://ciphersaber.gurus.com/faq.html#cs2
    j = 0
    for k in range(loops):
        for i in range(256):
            j = (j + sbox[i] + kbox[i]) % 256
            sbox[i], sbox[j] = sbox[j], sbox[i]

    # main loop
    i = 0
    j = 0
    out_bytes = bytearray()

    for car in in_bytes:
        i = (i + 1) % 256
        # [2] shuffle sbox
        j = (j + sbox[i]) % 256
        sbox[i], sbox[j] = sbox[j], sbox[i]
        # [3] compute t
        t = (sbox[i] + sbox[j]) % 256
        k = sbox[t]
        car = car ^ k
        out_bytes.append(car)

    return out_bytes

In [4]:
clear = bytearray(b'1234567890' * 100000)
t0 = time()
cipher = arcfour(b'key', clear)
print('elapsed time: %.2fs' % (time() - t0))
result = arcfour(b'key', cipher)
assert result == clear, '%r != %r' % (result, clear)
print('elapsed time: %.2fs' % (time() - t0))
print('OK')

elapsed time: 1.60s
elapsed time: 3.00s
OK


In [5]:
def crypto_process(size, key):
    in_text = bytearray(randrange(256) for i in range(size))
    cypher_text = arcfour(key, in_text)
    out_text = arcfour(key, cypher_text)
    assert in_text == out_text, 'Failed arcfour_test'
    return size
    
def main(workers=None):
    JOBS = 12
    SIZE = 2**18

    KEY = b"'Twas brillig, and the slithy toves\nDid gyre"
    STATUS = '{} workers, elapsed time: {:.2f}s'
    if workers:
        workers = int(workers)
    t0 = time()

    with ProcessPoolExecutor(workers) as executor:
        actual_workers = executor._max_workers
        to_do = []
        for i in range(JOBS, 0, -1):
            size = SIZE + int(SIZE / JOBS * (i - JOBS/2))
            job = executor.submit(crypto_process, size, KEY)
            to_do.append(job)

        for future in as_completed(to_do):
            res = future.result()
            print('{:.1f} KB'.format(res/2**10))

    print(STATUS.format(actual_workers, time() - t0))

In [6]:
main(1)

384.0 KB
362.7 KB
341.3 KB
320.0 KB
298.7 KB
277.3 KB
256.0 KB
234.7 KB
213.3 KB
192.0 KB
170.7 KB
149.3 KB
1 workers, elapsed time: 18.30s


In [7]:
main(2)

362.7 KB
384.0 KB
320.0 KB
341.3 KB
277.3 KB
298.7 KB
234.7 KB
256.0 KB
192.0 KB
213.3 KB
149.3 KB
170.7 KB
2 workers, elapsed time: 12.87s


In [8]:
main(4)

320.0 KB
341.3 KB
362.7 KB
384.0 KB
234.7 KB
256.0 KB
277.3 KB
298.7 KB
149.3 KB
170.7 KB
192.0 KB
213.3 KB
4 workers, elapsed time: 10.09s


#### MapReduce模型

这种模式可能更加被大家熟悉,同一个流程,将容器中的数据一条一脚放入子进程运算,最终也结果也会被放入容器中.最后可以将收集来的数据在主进程中进行处理

In [9]:
import math
PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]
def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

In [10]:
[is_prime(i) for i in PRIMES]

[True, True, True, True, True, False]

In [11]:
%timeit [is_prime(i) for i in PRIMES]

9.35 s ± 870 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def ProcessPool_prime(PRIMES= PRIMES ,workers=4):
    with ProcessPoolExecutor(max_workers=workers) as executor:
        total = []
        for prime in executor.map(is_prime, PRIMES):
            #print('%d is prime: %s' % (number, prime))
            total.append(prime)
    return total

In [13]:
ProcessPool_prime()

[True, True, True, True, True, False]

In [14]:
%timeit ProcessPool_prime(workers=4)

4.82 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 使用进程池进行相对底层的多进程操作

进程池的方式很适合批量创建子进程.

对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了。

请注意输出的结果，task 0，1，2，3是立刻执行的，而task 4要等待前面某个task完成后才执行，这是因为Pool的默认大小在我的电脑上是4，因此，最多同时执行4个进程。这是Pool有意设计的限制，并不是操作系统的限制。如果改成`p = Pool(5)`就可以同时跑5个进程。

由于Pool的默认大小是CPU的核数，如果你不幸拥有8核CPU，你要提交至少9个子进程才能看到上面的等待效果。

除了使用apply_async方法外,还有apply，map和map_async可以用于线程池的计算,编程模型也是如`concurrent.futures`一样分为两类

+ 并行任务模型
    + apply 单一任务布置
    + apply_async 非阻塞单一任务布置
+ MapReduce模型
    + map 同系统的map方法
    + map_async 非阻塞的map

#### apply_async

In [15]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('运行任务 %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('任务 %s 执行了 %0.2f 秒.' % (name, (end - start)))


In [16]:
print('父进程 %s.' % os.getpid())
p = Pool(4)
for i in range(5):
    p.apply_async(long_time_task, args=(i,))#创建非阻塞子进程用这个
print('等待所有子进程完成...')
p.close()
p.join()
print('所有子进程完成了.')

父进程 1372.
运行任务 0 (1432)...
运行任务 1 (1433)...
运行任务 2 (1434)...
运行任务 3 (1435)...
等待所有子进程完成...
任务 3 执行了 0.69 秒.
运行任务 4 (1435)...
任务 1 执行了 1.39 秒.
任务 4 执行了 0.94 秒.
任务 0 执行了 1.81 秒.
任务 2 执行了 2.42 秒.
所有子进程完成了.


#### map_async

In [17]:
from multiprocessing import Pool
from time import sleep

def f(x):
    return x*x

# start 4 worker processes
pool = Pool(processes=4)
print("map: ",pool.map(f, range(10)))
print("imap:")
for i in pool.imap_unordered(f, range(10)):
    print(i)

# evaluate "f(10)" asynchronously
res = pool.apply_async(f, [10])
print("apply:",res.get(timeout=1))             # prints "100"

# make worker sleep for 10 secs
res = pool.apply_async(sleep, [10])
print(res.get(timeout=1))             # raises multiprocessing.TimeoutError


map:  [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
imap:
0
1
4
9
16
25
36
64
49
81
apply: 100


TimeoutError: 

获取进程池中的运算结果

In [18]:
import multiprocessing
import time

def func(msg):
    print("msg:", msg)
    time.sleep(1)
    print("end")
    return "done " + msg


pool = multiprocessing.Pool(processes=4)
result = []
for i in range(3):
    msg = "hello %d" %(i)
    result.append(pool.apply_async(func, (msg, )))
pool.close()
pool.join()
for res in result:
    print(":::", res.get())
print("Sub-process(es) done.")

msg: hello 0
msg: hello 1
msg: hello 2
end
end
end
::: done hello 0
::: done hello 1
::: done hello 2
Sub-process(es) done.


## 更底层的多进程编程

标准库中的multiprocessing模块就是跨平台版本的多进程模块。

multiprocessing模块提供了一个Process类来代表一个进程对象，下面的例子演示了启动一个子进程并等待其结束：

In [19]:
from multiprocessing import Process
import os

#子进程要执行的代码
def run_proc(name):
    for i in range(3):
        print(u'子进程 %s (%s)...' % (name, os.getpid()))
    print(u'子进程结束.')

print(u'父进程 {}.'.format(os.getpid()))
p = Process(target=run_proc, args=('test',))
print(u'子进程要开始啦.')
p.start()
for i in range(3):
    print(u'父进程{pid}进行中...'.format(pid = os.getpid()))
p.join()
print(u"父进程结束啦")

父进程 1372.
子进程要开始啦.
子进程 test (1444)...
子进程 test (1444)...
子进程 test (1444)...
子进程结束.
父进程1372进行中...
父进程1372进行中...
父进程1372进行中...
父进程结束啦


### 使用Process作为父类自定义子进程

Process的子类需要重写`run`方法

In [20]:
from multiprocessing import Process, Queue
class Processor(Process):

    def __init__(self, queue, idx):
        super(Processor, self).__init__()
        self.queue = queue
        self.idx = idx

    def return_name(self):
        ## NOTE: self.name is an attribute of multiprocessing.Process
        return "Process idx=%s is called '%s'" % (self.idx, self.name)

    def run(self):
        self.queue.put(self.return_name())


In [21]:
processes = list()
q = Queue()
for i in range(0,5):
    p=Processor(queue=q, idx=i)
    processes.append(p)
    p.start()
for proc in processes:
    proc.join()
    ## NOTE: You cannot depend on the results to queue / dequeue in the
    ## same order
    print("RESULT: {}".format(q.get()))

RESULT: Process idx=0 is called 'Processor-57'
RESULT: Process idx=1 is called 'Processor-58'
RESULT: Process idx=2 is called 'Processor-59'
RESULT: Process idx=3 is called 'Processor-60'
RESULT: Process idx=4 is called 'Processor-61'


创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，这样创建进程比fork()简单。

join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。

可以看到我们的父进程进行完了子进程才进行.其实当执行start方法的时候我们就已经把进程创建好并给他任务了. 虽然进程启动了,但我们并不能知道它啥时候运算完成.这时候用join方法来确认是否执行完了(通过阻塞主进程),也就是起个等待结果的作用.

## 进程间通信

如何让进程间通信呢,其实原理上来讲就是构造一个独立的数据结构来存放结果来参与通信

有两种方式,最常用的一种是用队列

### 先进先出队列Queue

In [22]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

q = Queue()
p = Process(target=f, args=(q,))
p.start()
print(q.get())    # prints "[42, None, 'hello']"
p.join()

[42, None, 'hello']


个稍微复杂一点的例子:

In [23]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())
# 读数据进程执行的代码:
def read(q):
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    while True:
        if not q.empty():
            value = q.get(True)
            print('Get %s from queue.' % value)
            time.sleep(random.random())
        else:
            q.put("Done!")
            break

# 父进程创建Queue，并传给各个子进程：
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# 启动子进程pw，写入:
pw.start()
# 等待pw结束:
pw.join()
# 启动子进程pr，读取:
pr.start()
pr.join()
print(q.get())
print('\n所有数据都写入并且读完')

Put A to queue...
Put B to queue...
Put C to queue...
Get A from queue.
Get B from queue.
Get C from queue.
Done!

所有数据都写入并且读完


两个进程间,父进程创建一个队列给各个子进程,子进程接收父进程的队列作为参数运行. 运行过程中将结果存入队列最后运行完后将”done!”存入队列,由父进程接收.

### 生产者消费者模型

队列最常见的用处就是在生产者消费者模式中作为数据缓冲区.以下就是一个生产者消费者模式的例子

In [24]:
import random
from multiprocessing import Process, Queue,Condition

In [25]:
class Producer(Process):
    """生产者"""
    def __init__(self,q,con,name):
        super(Producer,self).__init__()
        self.q = q
        self.name = name
        self.con = con
        print("生产者{self.name}产生了".format(self=self))

    def run(self):
        count = 3 #只生产满3轮,要不然就会无限循环出不去了
        while count>0:
            #global writelock
            self.con.acquire()
            if self.q.full():
                print("队列满了,生产者等待")
                count-=1
                self.con.wait()

            else:
                value = random.randint(0,10)
                print("{self.name}把值{self.name}:{value}放入了队列".format(self=self,value=value))
                self.q.put("{self.name}:{value}".format(self=self,value=value))
            self.con.notify()
        self.con.release()

In [26]:
class Consumer(Process):
    """消费者"""
    def __init__(self,q,con,name):
        super(Consumer,self).__init__()
        self.q = q
        self.name = name
        self.con = con
        print("消费者{self.name}产生了".format(self=self))

    def run(self):
        while True:
            #global writelock
            self.con.acquire()
            if self.q.empty():

                print("队列空了,消费者等待")
                self.con.wait()
            else:
                value = self.q.get()

                print("{self.name}从队列中获取了{self.name}:{value}".format(self=self,
                                                                         value=value))
                self.con.notify()
            self.con.release()

In [27]:
q = Queue(10)
con = Condition()
p1 = Producer(q,con,"P1")
p1.start()
p2 = Producer(q,con,"P2")
p2.start()
c1 = Consumer(q,con,"C1")
c1.start()

生产者P1产生了
生产者P2产生了
消费者C1产生了
P1把值P1:1放入了队列
P1把值P1:2放入了队列
P1把值P1:9放入了队列
P1把值P1:1放入了队列
P1把值P1:8放入了队列
P1把值P1:10放入了队列
P1把值P1:5放入了队列
P1把值P1:8放入了队列
P1把值P1:6放入了队列
P1把值P1:9放入了队列
队列满了,生产者等待
队列满了,生产者等待
C1从队列中获取了C1:P1:1
P1把值P1:2放入了队列
队列满了,生产者等待
C1从队列中获取了C1:P1:2
P2把值P2:7放入了队列
队列满了,生产者等待
队列满了,生产者等待
C1从队列中获取了C1:P1:9
P2把值P2:0放入了队列
队列满了,生产者等待


### 管道Pipes

既然是管道,那就肯定有两端,有方向,分成单向管道和双向管道了.

看一个最简单的双向管道

In [28]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()


parent_conn, child_conn = Pipe()
p = Process(target=f, args=(child_conn,))
p.start()
print(parent_conn.recv())   # prints "[42, None, 'hello']"
p.join()

[42, None, 'hello']


稍微复杂的例子:

In [29]:
from multiprocessing import Process, Pipe
import os, time, random

# 写数据进程执行的代码:
def write(conn):
    value = ["h1 reader~"]
    print('Put %s to pip...' % value)
    conn.send(value)
    time.sleep(1)

# 读数据进程执行的代码:
def read(conn):
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    value = conn.recv()
    print('Get %s from pip.' % value)
    conn.send("hi writer~~")



# 父进程创建Pipe，并传给各个子进程：
parent_conn, child_conn = Pipe()
pw = Process(target=write, args=(parent_conn,))#起点
pr = Process(target=read, args=(child_conn,))#终点
# 启动子进程pw，写入:
pw.start()
# 等待pw结束:
pw.join()
# 启动子进程pr，读取:
pr.start()
pr.join()
print(parent_conn.recv())
print('\n所有数据都写入并且读完')

Put ['h1 reader~'] to pip...
Get ['h1 reader~'] from pip.
hi writer~~

所有数据都写入并且读完


可以看出管道的限制相对多些,必须要建立连接才能交换数据,一出一进这样子,这也是为啥队列用的比较多.

## 静态数据共享


python里面的全局变量也只管到他自己的进程,如果要让一个静态的数据在每个子进程中都可以调用.那么需要用到模块中的几个方法:

+ Value, Array

    静态数据位共享,静态数组共享,本质就是在内存中开辟一块用于共享的空间,Value和Array都必须使用C类型保存数据

In [30]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]


num = Value('d', 0.0)
arr = Array('i', range(10))

p = Process(target=f, args=(num, arr))
p.start()
p.join()

print(num.value)
print(arr[:])

3.1415927
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]


### 高级共享multiprocessing.Manager

之前介绍了queue,pipe,array和value,这些都太具体底层,有没有什么方法可以像处理python容器一样
简单地处理数据共享的问题呢?multiprocess提供一个manager模块.

Manager()返回的manager对象控制了一个server进程，此进程包含的python对象可以被其他的进程通过proxies来访问。从而达到多进程间数据通信且安全。

Manager支持的类型有

+ list
+ dict
+ Namespace
+ Lock
+ RLock
+ Semaphore
+ BoundedSemaphore
+ Condition
+ Event
+ Queue
+ Value
+ Array。



In [31]:
import multiprocessing
import time

def worker(d, key, value):
    d[key] = value


mgr = multiprocessing.Manager()
d = mgr.dict()
jobs = [ multiprocessing.Process(target=worker, args=(d, i, i*2))
         for i in range(10)
         ]
for j in jobs:
    j.start()
for j in jobs:
    j.join()
print ('Results:' )
for key, value in enumerate(dict(d)):
    print("%s=%s" % (key, value))

Results:
0=0
1=1
2=2
3=3
4=4
5=5
6=6
7=7
8=8
9=9


namespace对象没有公共的方法，但是有可写的属性

In [32]:
import multiprocessing
manager = multiprocessing.Manager()
Global = manager.Namespace()
Global.x = 10
Global.y = 'hello'
print(Global)

Namespace(x=10, y='hello')
